In [1]:
#to suppress all tensorflow warnings
import warnings
warnings.filterwarnings("ignore")
from os import environ, chdir
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import matplotlib 
import numpy 
import piexif
import keras
import os 
import random
import shutil
from matplotlib import pyplot as plt 

#Model Building Libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dropout, Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

Using TensorFlow backend.


1)75:25 Training and Test ratio is used as it yielded best Testing accuracy
2)Since Data Augmentation yields better results we are only using Data Augmented Images 
3)10 Epochs for Training  
4)Train and Test are already loaded with 75:25 ratio images.so we are not redoing the images split and directly accessing the train and test folder respectively

In [29]:
#Split ratio 
train_size = 0.75

_,_, fresh_images = next(os.walk('C:/Users/Deepika/CS5000/Model_Transfer_Learning/Dataset/AppleImages/fresh'))
_,_, rotten_images = next(os.walk('C:/Users/Deepika/CS5000/Model_Transfer_Learning/Dataset/AppleImages/rotten'))

#Total number of fresh and rotten images
num_fresh_images = len(fresh_images)
num_rotten_images = len(rotten_images)

#Split fresh images into training images and test images with given split ratio
num_fresh_images_train = int(train_size * num_fresh_images)
num_fresh_images_test = num_fresh_images - num_fresh_images_train

#Split rotten images into training images and test images with given split ratio
num_rotten_images_train = int(train_size * num_rotten_images)
num_rotten_images_test = num_rotten_images - num_rotten_images_train

print("Number of training fresh images",num_fresh_images_train)
print("Number of training rotten images",num_rotten_images_train)
print("Number of Training images ",num_fresh_images_train+num_rotten_images_train)
print("********************************************************************")
print("Number of testing fresh images",num_fresh_images_test)
print("Number of testing rotten images",num_rotten_images_test)
print("Number of Testing images ",num_fresh_images_test+num_rotten_images_test )

Number of training fresh images 1566
Number of training rotten images 2207
Number of Training images  3773
********************************************************************
Number of testing fresh images 522
Number of testing rotten images 736
Number of Testing images  1258


In [30]:
#Basic hyperparameters or the original Hyperparameters
FILTER_SIZE = 3 #best size -same as before
NUM_FILTERS = 32 #best count -same as beore
INPUT_SIZE  = 64
MAXPOOL_SIZE = 3  
BATCH_SIZE = 32 #best batch size found
EPOCHS = 10 


#Steps per epoch is based on Training so in total we have 1670 fresh and 2354 rotten images =4024/batch size=125.75 =>126
STEP_PER_TRAIN = round((num_rotten_images_train+num_fresh_images_train)/BATCH_SIZE)
STEP_PER_TEST= round((num_rotten_images_test+num_fresh_images_test)/BATCH_SIZE)

print("Training step size is {0},Testing step size {1}".format(STEP_PER_TRAIN,STEP_PER_TEST))

Training step size is 118,Testing step size 39


In [31]:
#Image Generator with Data Augmentation
training_data_generator = ImageDataGenerator(rescale = 1./255,
                                             shear_range = 0.2,
                                             zoom_range = 0.2,
                                             horizontal_flip = True)#AUGMENTATION DONE HERE SINCE THIS IS TRAINING IMAGE

training_set = training_data_generator.flow_from_directory('C:/Users/Deepika/CS5000/Model_Transfer_Learning/Dataset/AppleImages/Train/',
                                                           target_size=(INPUT_SIZE,INPUT_SIZE),                                                    
                                                           batch_size=BATCH_SIZE,                                                    
                                                           class_mode='binary')

testing_data_generator = ImageDataGenerator(rescale = 1./255)#NO MUCH AUGMENTATION DONE HERE-NO CHANGE

test_set = testing_data_generator.flow_from_directory('C:/Users/Deepika/CS5000/Model_Transfer_Learning/Dataset/AppleImages/Test/',                               
                                                      target_size=(INPUT_SIZE,INPUT_SIZE),                        
                                                      batch_size=BATCH_SIZE,                                   
                                                      class_mode = 'binary')

Found 3773 images belonging to 2 classes.
Found 1258 images belonging to 2 classes.


In [35]:
#Remove TF warnings
warnings.filterwarnings("ignore")
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#Model4 (conv+relu+maxpool)(conv+relu+maxpool)(Dense 128)(Dense 1)
classifier_adam = Sequential() 

classifier_adam.add(Conv2D(NUM_FILTERS, (FILTER_SIZE, FILTER_SIZE),  
                 input_shape = (INPUT_SIZE, INPUT_SIZE, 3),                
                 activation = 'relu')) 

#we add a max pooling layer:
classifier_adam.add(MaxPooling2D(pool_size = (MAXPOOL_SIZE, MAXPOOL_SIZE))) 

#Repeat 2nd convolutional layer
classifier_adam.add(Conv2D(NUM_FILTERS, (FILTER_SIZE, FILTER_SIZE),  
                 input_shape = (INPUT_SIZE, INPUT_SIZE, 3),                
                 activation = 'relu')) 

#we add a max pooling layer:
classifier_adam.add(MaxPooling2D(pool_size = (MAXPOOL_SIZE, MAXPOOL_SIZE))) 

#flatten multi dimensional array to single to 1dimensional vector
classifier_adam.add(Flatten())

#add a fully connected layer with 128 nodes:
classifier_adam.add(Dense(units = 128, activation = 'relu'))

#one last fully connected layer to our Model:
classifier_adam.add(Dense(units = 1, activation = 'sigmoid')) 

classifier_adam.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 

classifier_adam.summary()

#Fit model for epochs=10 
history_adam=classifier_adam.fit_generator(training_set, 
                    steps_per_epoch = STEP_PER_TRAIN,
                    epochs=EPOCHS 
                    )

#Evaluate model using Validation set
loss,acc=classifier_adam.evaluate_generator(generator=test_set,steps=STEP_PER_TEST)
print("Loss :{0} and Accuracy: {1}".format(loss,acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               147584    
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 129       
Total para

In [36]:
#new dataset for 0.75 dataset
#maxpool size 3
classifier_adam.save('Fruit_status_final_model_0.75_97.92.h5')

In [49]:
classifier_adam.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               147584    
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 129       
Total para

In [50]:
classifier_adam.save("Fruit_Status_97_92.h5")